# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
import csv
import json
#
# import API key
from api_keys import g_key

import requests
import collections
collections.Iterable = collections.abc.Iterable




### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# load the csv, old data
data = pd.read_csv("/Users/siyuanliang/BootCamp_University/Python-api-challenge/WeatherPy/Whether.csv")
data


,Unnamed: 0,City_ID,City,City Country,City date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,1,malayal,PH,1665498930,7.2080,121.9420,79.57,84,97,3.29
1,1,2,fortuna,US,1665498931,40.5982,-124.1573,52.66,88,100,4.61
2,2,3,sao filipe,CV,1665498931,14.8961,-24.4956,80.37,74,26,7.65
3,3,4,arraial do cabo,BR,1665498931,-22.9661,-42.0278,78.66,78,0,14.97
4,4,5,igarka,RU,1665498932,67.4667,86.5833,35.22,97,100,15.55
...,...,...,...,...,...,...,...,...,...,...,...
531,531,532,campbell river,CA,1665499076,50.0163,-125.2446,50.27,66,20,6.91
532,532,533,athabasca,CA,1665499076,54.7169,-113.2854,36.01,39,0,12.08
533,533,534,semme,FR,1665499076,46.1963,1.5007,61.21,88,100,1.77
534,534,535,skjervoy,NO,1665499077,70.0311,20.9714,50.00,76,14,3.98


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
gmaps.configure(api_key=g_key)
# Create a list containing coordinates
locations = data[["Lat", "Lng"]]
humidity = data["Humidity"]

# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, center=(55.0, -1.0), zoom_level=5)

# Use the Lat and Lng as locations and Humidity as the weight.
# Add the weighted heat map layer to the map
fig.add_layer(gmaps.heatmap_layer(locations, weights=humidity, max_intensity=100))

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#  narrow DataFrame
Newdata = data.loc[(data['Max Temp'] <= 80) & (data['Max Temp'] > 70) & (data['Wind Speed']<10) & (data["Cloudiness"]==0)]
Newdata

,Unnamed: 0,City_ID,City,City Country,City date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
23,23,24,muros,ES,1665498909,42.7762,-9.0603,76.84,63,0,6.06
71,71,72,kapaa,US,1665498949,22.0752,-159.3190,76.98,83,0,9.22
94,94,95,laguna,US,1665498956,38.4210,-121.4238,70.77,79,0,6.91
150,150,151,ascension,US,1665498971,30.2169,-90.9001,75.20,56,0,4.61
193,193,194,antsohihy,MG,1665498984,-14.8796,47.9875,78.64,58,0,7.61
293,293,294,dalbandin,PK,1665499014,28.8947,64.4101,74.64,7,0,5.97
331,331,332,makakilo city,US,1665499023,21.3469,-158.0858,75.56,87,0,3.44
356,356,357,beihai,CN,1665499030,21.4833,109.1000,70.93,48,0,9.95
408,408,409,ambilobe,MG,1665499042,-13.2000,49.0500,76.12,57,0,4.41
415,415,416,kazerun,IR,1665498834,29.6195,51.6541,79.66,25,0,4.65


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Create new plot
# create a heatmap based on the csv


# generate some (latitude, longitude) pairs
fig_hotel_humidity = gmaps.figure()
fig_hotel_humidity.add_layer(gmaps.heatmap_layer(Newdata[["Lat", "Lng"]], weights=Newdata["Humidity"], max_intensity=100))
fig_hotel_humidity

Figure(layout=FigureLayout(height='420px'))

In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{City Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

# get hotel info using Google Place api
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# set parameters to search for hotels
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe

for index, row in Newdata.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # extract results
    results = response['results']
    # save the hotel name to dataframe
    try:
        print(f"First hotel in {row['City']} is {results[0]['name']}.")

        # store hotel name
        Newdata.loc[index, 'Hotel Name'] = results[0]['name']

        # store hotel coordinates
        Newdata.loc[index, 'Hotel Lat'] = results[0]['geometry']['location']['lat']
        Newdata.loc[index, 'Hotel Lng'] = results[0]['geometry']['location']['lng']


    except (KeyError, IndexError):
        # set empty value
        Newdata.loc[index, 'Hotel Name'] = ""
        Newdata.loc[index, 'Hotel Lat'] = ""
        Newdata.loc[index, 'Hotel Lng'] = ""

        print("Missing field/result... skipping.")
    print("------------")

Newdata

Retrieving Results for Index 23: muros.
First hotel in muros is Apartment Av. Castelao.
------------
Retrieving Results for Index 71: kapaa.


/var/folders/t0/trftcgf923z_mljxn6xk43hw0000gn/T/ipykernel_65511/2877025347.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Newdata.loc[index, 'Hotel Name'] = results[0]['name']
/var/folders/t0/trftcgf923z_mljxn6xk43hw0000gn/T/ipykernel_65511/2877025347.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Newdata.loc[index, 'Hotel Lat'] = results[0]['geometry']['location']['lat']
/var/folders/t0/trftcgf923z_mljxn6xk43hw0000gn/T/ipykernel_65511/2877025347.py:45: SettingWithCopyWarning: 
A value is tryin

First hotel in kapaa is Sheraton Kauai Coconut Beach Resort.
------------
Retrieving Results for Index 94: laguna.
First hotel in laguna is Holiday Inn Express & Suites Elk Grove Central - Hwy 99, an IHG Hotel.
------------
Retrieving Results for Index 150: ascension.
First hotel in ascension is Highland Inn.
------------
Retrieving Results for Index 193: antsohihy.
First hotel in antsohihy is Hotel de Ville.
------------
Retrieving Results for Index 293: dalbandin.
First hotel in dalbandin is Sanjrani House.
------------
Retrieving Results for Index 331: makakilo city.
First hotel in makakilo city is Marriott's Ko Olina Beach Club.
------------
Retrieving Results for Index 356: beihai.
First hotel in beihai is 21 Degree Hostel.
------------
Retrieving Results for Index 408: ambilobe.
First hotel in ambilobe is Hôtel & Restaurant DIANA Ambilobe.
------------
Retrieving Results for Index 415: kazerun.
First hotel in kazerun is آپارتمان بزرگ اویسا.
------------
Retrieving Results for Ind

,Unnamed: 0,City_ID,City,City Country,City date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name,Hotel Lat,Hotel Lng
23,23,24,muros,ES,1665498909,42.7762,-9.0603,76.84,63,0,6.06,Apartment Av. Castelao,42.775733,-9.058663
71,71,72,kapaa,US,1665498949,22.0752,-159.3190,76.98,83,0,9.22,Sheraton Kauai Coconut Beach Resort,22.056623,-159.322447
94,94,95,laguna,US,1665498956,38.4210,-121.4238,70.77,79,0,6.91,Holiday Inn Express & Suites Elk Grove Central...,38.424083,-121.398707
150,150,151,ascension,US,1665498971,30.2169,-90.9001,75.20,56,0,4.61,Highland Inn,30.211274,-90.939170
193,193,194,antsohihy,MG,1665498984,-14.8796,47.9875,78.64,58,0,7.61,Hotel de Ville,-14.881228,47.988075
293,293,294,dalbandin,PK,1665499014,28.8947,64.4101,74.64,7,0,5.97,Sanjrani House,28.898438,64.402688
331,331,332,makakilo city,US,1665499023,21.3469,-158.0858,75.56,87,0,3.44,Marriott's Ko Olina Beach Club,21.333048,-158.120272
356,356,357,beihai,CN,1665499030,21.4833,109.1000,70.93,48,0,9.95,21 Degree Hostel,21.484125,109.106061
408,408,409,ambilobe,MG,1665499042,-13.2000,49.0500,76.12,57,0,4.41,Hôtel & Restaurant DIANA Ambilobe,-13.204478,49.031226
415,415,416,kazerun,IR,1665498834,29.6195,51.6541,79.66,25,0,4.65,آپارتمان بزرگ اویسا,29.638432,51.667280


In [8]:
# filter out cities with no hotel# filter out cities with no hotel
Newdata = Newdata.loc[Newdata['Hotel Name'] != ""]

info_content = [info_box_template.format(**row) for index, row in Newdata.iterrows()]

info_content

# Add hotel marker layer on top of heat map using info box template
fig_hotel_humidity.add_layer(gmaps.marker_layer(Newdata[["Hotel Lat", "Hotel Lng"]], info_box_content=info_content))
fig_hotel_humidity


Figure(layout=FigureLayout(height='420px'))